# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved. 
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server. 

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

# Imports

In [51]:
from gensim.models import Word2Vec
import numpy as np
from string import ascii_lowercase as alc
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from numpy.linalg import norm
from tqdm import tqdm

In [3]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections

from urllib.parse import parse_qs, urlencode, urlparse

import warnings
warnings.filterwarnings("ignore")

# Load Words

In [4]:
## Load words
train_words_location = "../data/words_250000_train.txt"
with open(train_words_location, "r") as fp:
    train_word_list = fp.read().splitlines()
for i in range(len(train_word_list)):
    train_word_list[i] = train_word_list[i].strip().lower()
train_word_list = sorted(list(set(train_word_list)))

In [5]:
print(len(train_word_list))

227300


# Prepare data for char mebeddings

In [11]:
char_sequence_list = [list(set(word)) for word in train_word_list if(len(set(word))>2)]

In [13]:
char_sequence_list[:2], len(char_sequence_list)

([['c', 'n', 'h', 'e', 'a'], ['d', 'e', 'a', 'h']], 226644)

# Learn character embeddings


In [15]:
char2vec_model = Word2Vec(
    sentences=char_sequence_list, 
    vector_size=128,
    window=10,
    min_count=2,
    workers=6,
    sg=1,
    hs=0,
    negative=20,
    ns_exponent=0.75,
    alpha=0.025,
    min_alpha=0.0001,
    seed=42,
    sample=1e-3,
    hashfxn=hash, 
    epochs=10,
    sorted_vocab=1, 
    null_word=0,
    compute_loss=True,
    shrink_windows=True
)

In [16]:
char_vectors = char2vec_model.wv

# Check char embeddings

In [17]:
len(char_vectors)

26

In [46]:
guessed_letters = ["h", "p", "y"]
context_letters = ["h", "p", "y"]
cos_sim_dict = {char_: 0 for char_ in alc if(char_ not in guessed_letters)}
for char_ in cos_sim_dict.keys():
    char_vector = char_vectors[char_]
    for context_letter in context_letters:
        context_letter_vector = char_vectors[context_letter]
        sim = np.dot(context_letter_vector, char_vector)/(norm(context_letter_vector) * norm(char_vector))
        cos_sim_dict[char_] = max(cos_sim_dict[char_], sim)
    

In [47]:
sorted_sim = sorted(cos_sim_dict.items(), key=lambda x:x[1])
print(sorted_sim)

[('m', 0.1896882), ('c', 0.19718611), ('w', 0.20646392), ('g', 0.23130614), ('k', 0.25566292), ('t', 0.2617181), ('d', 0.27489796), ('l', 0.27625036), ('i', 0.2763598), ('n', 0.29479197), ('u', 0.29678738), ('x', 0.29737744), ('f', 0.30145884), ('s', 0.3104473), ('b', 0.31069723), ('a', 0.31371987), ('o', 0.3420106), ('r', 0.34381992), ('z', 0.35762507), ('e', 0.38338003), ('q', 0.40159112), ('j', 0.40368915), ('v', 0.4552739)]


# Given Code

In [48]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        
        full_dictionary_location = "../data/words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        self.current_dictionary = []
        
    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def guess(self, word): # word input example: "_ p p _ e "
        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        clean_word = word[::2].replace("_",".").lower()

        # Get list of context letters
        context_letter_list = [c for c in clean_word if(c!=".")]

        # Get guessed letter list
        guessed_letter_list = self.guessed_letters
        
        # Initialize output
        guess_letter = '!'
        
        # Get most similar letter based on the context letters
        unguessed_letter_list = [c for c in alc if(c not in guessed_letter_list)]
        max_cos_sim = -np.inf
        if(len(context_letter_list) > 0):
            for c in unguessed_letter_list:
                c_vector = char_vectors[c]
                c_cos_sim_list = []
                for context_letter in context_letter_list:
                    context_letter_vector = char_vectors[context_letter]
                    cos_sim = np.dot(context_letter_vector, c_vector)/(norm(context_letter_vector) * norm(c_vector))
                    c_cos_sim_list.append(cos_sim)
                c_cos_sim_max = max(c_cos_sim_list)
                if(c_cos_sim_max > max_cos_sim):
                    guess_letter = c
                    max_cos_sim = c_cos_sim_max
        
        # if no word matches in training dictionary, default back to ordering of full dictionary
        if guess_letter == '!':
            sorted_letter_count = self.full_dictionary_common_letter_sorted
            for letter,instance_count in sorted_letter_count:
                if letter not in guessed_letter_list:
                    guess_letter = letter
                    break
        
        return guess_letter

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [49]:
token = "16961edd52e8d62c2cf18e3bdecf31"
api = HangmanAPI(access_token=token, timeout=2000)

## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [52]:
for i in tqdm(range(10)):
    api.start_game(practice=1,verbose=False)
    time.sleep(0.5)

100%|██████████| 10/10 [00:54<00:00,  5.45s/it]


In [54]:
api.start_game(practice=1,verbose=True)

Successfully start a new game! Game ID: 9369e24c8d95. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Sever response: {'game_id': '9369e24c8d95', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ _ _ _ e '}
Guessing letter: y
Sever response: {'game_id': '9369e24c8d95', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ _ _ _ e '}
Guessing letter: z
Sever response: {'game_id': '9369e24c8d95', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ _ _ _ _ _ _ e '}
Guessing letter: p
Sever response: {'game_id': '9369e24c8d95', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ _ _ _ _ _ _ _ e '}
Guessing letter: g
Sever response: {'game_id': '9369e24c8d95', 'status': 'ongoing', 'tries_remains': 2, 'word': '_ _ _ _ _ _ _ _ e '}
Guessing letter: j
Sever response: {'game_id': '9369e24c8d95', 'status': 'ongoing', 'tries_remains': 1, 'word': '_ _ _ _ _ _ _ _ e '}
Guessing letter: f
Sever response: {'game_id': '9369e24c8d95', 'status': 'ongoing', 'tries_

False

In [53]:
# api.start_game(practice=1,verbose=True)
# Get my game stats: (# of tries, # of wins)
[total_practice_runs, total_recorded_runs, total_recorded_successes, total_practice_successes] = api.my_status()
practice_success_rate = total_practice_successes / total_practice_runs
print(f"Total Practice Runs: {total_practice_runs}")
print(f"Total Practice Success: {total_practice_successes}")
print(f"Overall Practice Success Rate: {practice_success_rate}")

Total Practice Runs: 122
Total Practice Success: 50
Overall Practice Success Rate: 0.4098360655737705


## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [ ]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    #api.start_game(practice=0,verbose=False)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [ ]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)